In [1]:
# Load Data and HMM scripts
import torch
import pandas as pd
import numpy as np
import pickle
import HMM_Likelyhood
import plotly.express as px

with open('DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = X_tensor = torch.from_numpy(X).to(torch.float64)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)


def Accumulated_HHM_LL(X: Tensor,
    Pinit: Tensor,
    kappa: Tensor,
    mu: Tensor,
    Tk: Tensor,
    n: int,
    p: int,
    K: int) -> Tensor:
  _0 = __torch__.HMM_Likelyhood.HMM_log_likelihood
  kappa_con = torch.minimum(^<python_value>()(kappa), torch.tensor([800]))
  _1 = torch.sqrt(torch.sum(torch.pow(mu, 2), [0]))
  mu_con = torch.div(mu, _1)
  Subjectlog_Likelihood = torch.zeros([n])
  for subject in range(n):
    _2 = torch.slice(X)
    _3 = torch.mul(330, subject)
    _4 = torch.mul(330, torch.add(subject, 1))
    _5 = torch.slice(_2, 1, _3, _4)
    _6 = torch.select(torch.slice(Pinit), 1, subject)
    _7 = ^<python_value>()(_6)
    _8 = ^<python_value>()(torch.select(Tk, 0, subject))
    _9 = _0(_5, _7, kappa_con, mu_con, _8, p, K, )
    _10 = torch.select(Subjectlog_Likelihood, 0, subject)
    _11 = torch.copy_(_10, _9)
  return torch.sum(Subjectlog_Likelihood)



In [2]:
# Setup data for Fitting
Repetetions = 5
Subjects = 20
Likelihoods = np.zeros((5,19))
Xtrain = torch.zeros((90,165*Subjects))
Xtest = torch.zeros((90,165*Subjects))
for i in range(Subjects):
    Xtrain[:,i*(165):((i+1)*165)] = X_tensor[:,i*(330):((i*330+165))]
    Xtest[:,i*(165):((i+1)*165)] = X_tensor[:,i*(330)+165:((i*330+330))]

lr= 1.2
n_iters=1000

In [7]:
# Loop over all K's to be evaluated
for idx,K in enumerate(range(1,16)):
    for rep in range(Repetetions):
            kappa,mu,Tk,Pinit = HMM_Likelyhood.InitializeParametersFF(X_tensor,Subjects,p=90,K=K)

            Parameters = [
                {'params':Pinit},
                {'params':kappa},
                {'params':mu},
                {'params':Tk},
            ]

            Adam = torch.optim.Adam(Parameters,lr=lr)
            HMM_Likelyhood.OptimizationTraj(Xtrain,[Pinit,kappa,mu,Tk],lose=HMM_Likelyhood.Accumulated_HHM_LLHalf,Optimizer=Adam,n=Subjects,n_iters=n_iters,K=K)

            Likelihoods[rep,K-2] = HMM_Likelyhood.Accumulated_HHM_LLHalf(Xtest,Pinit,kappa,mu,Tk,n=Subjects,p=90,K=K)

epoch 1; Log-Likelihood = -241355.07676478272
epoch 101; Log-Likelihood = -307795.64725452114
epoch 201; Log-Likelihood = -316093.08925743366
epoch 301; Log-Likelihood = -326623.6857789706
epoch 401; Log-Likelihood = -337269.22867965803
epoch 501; Log-Likelihood = -347851.98984003474
epoch 601; Log-Likelihood = -357233.1812167928
epoch 701; Log-Likelihood = -365964.1567597541
epoch 801; Log-Likelihood = -374183.6240303109
epoch 901; Log-Likelihood = -379781.3561943339
epoch 1000; Log-Likelihood = -380227.71817153285
epoch 1; Log-Likelihood = -241364.42196062612
epoch 101; Log-Likelihood = -311497.7848102171
epoch 201; Log-Likelihood = -319636.1436369316
epoch 301; Log-Likelihood = -329518.09883851476
epoch 401; Log-Likelihood = -341140.294809267
epoch 501; Log-Likelihood = -355193.80690752156
epoch 601; Log-Likelihood = -370123.69981088006
epoch 701; Log-Likelihood = -378858.766466527
epoch 801; Log-Likelihood = -381507.20237024396
epoch 901; Log-Likelihood = -383973.4549560844
epoch 1

In [17]:
# Save Likelihoods
with open('HMM_K_Eval.pickle','wb') as f:
    pickle.dump([Likelihoods], f)